In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
assert sys.version_info.major == 3
import os

add_paths = True
if add_paths:
  sys.path.insert(0, os.path.join(os.path.abspath(os.getcwd()), '..', '..'))
  sys.path.insert(
      0,
      os.path.join(os.path.abspath(os.getcwd()), '..', '..', 'build', 'python'))
  import pyspiel


from open_spiel.python.algorithms import cfr
from open_spiel.python.algorithms import exploitability
from open_spiel.python.algorithms import expected_game_score
from open_spiel.python.bots import uniform_random
from open_spiel.python.visualizations import treeviz

In [2]:
games_list = pyspiel.registered_names()

print("Registered games:")
print(games_list)

game = pyspiel.load_game("universal_poker")

Registered games:
['backgammon', 'battleship', 'blackjack', 'blotto', 'breakthrough', 'bridge', 'bridge_uncontested_bidding', 'catch', 'chess', 'cliff_walking', 'clobber', 'coin_game', 'connect_four', 'coop_box_pushing', 'coop_to_1p', 'coordinated_mp', 'cursor_go', 'dark_chess', 'dark_hex', 'dark_hex_ir', 'deep_sea', 'efg_game', 'first_sealed_auction', 'gin_rummy', 'go', 'goofspiel', 'hanabi', 'havannah', 'hearts', 'hex', 'kriegspiel', 'kuhn_poker', 'laser_tag', 'leduc_poker', 'lewis_signaling', 'liars_dice', 'liars_dice_ir', 'markov_soccer', 'matching_pennies_3p', 'matrix_cd', 'matrix_coordination', 'matrix_mp', 'matrix_pd', 'matrix_rps', 'matrix_rpsw', 'matrix_sh', 'matrix_shapleys_game', 'mfg_crowd_modelling', 'mfg_crowd_modelling_2d', 'misere', 'negotiation', 'nfg_game', 'normal_form_extensive_game', 'oh_hell', 'oshi_zumo', 'othello', 'oware', 'pentago', 'phantom_ttt', 'phantom_ttt_ir', 'pig', 'quoridor', 'rbc', 'repeated_game', 'sheriff', 'skat', 'solitaire', 'start_at', 'stones_a

In [3]:
"""Test that Python and C++ bots can be called by a C++ algorithm."""

from absl.testing import absltest
import numpy as np

from open_spiel.python.bots import uniform_random

game = pyspiel.load_game("leduc_poker")
bots = [
    pyspiel.make_uniform_random_bot(0, 1234),
    uniform_random.UniformRandomBot(1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
leduc_average_results = np.mean(results, axis=0)
print(leduc_average_results)

game = pyspiel.load_game("universal_poker")
bots = [
    pyspiel.make_uniform_random_bot(0, 1234),
    uniform_random.UniformRandomBot(1, np.random.RandomState(4321)),
]
results = np.array([
    pyspiel.evaluate_bots(game.new_initial_state(), bots, iteration)
    for iteration in range(10000)
])
universal_poker_average_results = np.mean(results, axis=0)
print(universal_poker_average_results)

#np.testing.assert_allclose(universal_poker_average_results, leduc_average_results, atol=0.1)

[-0.1391  0.1391]
[ 22.73 -22.73]


In [4]:
universal_poker_kuhn_limit_3p = """\
GAMEDEF
betting = limit
numPlayers = 3
numRounds = 1
blind = 1 1 1
raiseSize = 1
firstPlayer = 1
maxRaises = 1
numSuits = 1
numRanks = 4
numHoleCards = 1
numBoardCards = 0
END GAMEDEF
"""

game = pyspiel.load_game(
    "universal_poker",
    {"gamedef": universal_poker_kuhn_limit_3p})
str(game)

'universal_poker(gamedef=GAMEDEF\nlimit\nnumPlayers = 3\nnumRounds = 1\nblind = 1 1 1\nraiseSize = 1\nfirstPlayer = 1\nmaxRaises = 1\nnumSuits = 1\nnumRanks = 4\nnumHoleCards = 1\nnumBoardCards = 0\nEND GAMEDEF\n)'

In [5]:
# Compare exloitability for two games
players = 2
iterations = 10
print_freq = 1

def compare_exploitability(game_1, game_2):
  cfr_solver_1 = cfr.CFRSolver(game_1)
  cfr_solver_2 = cfr.CFRSolver(game_2)
  for i in range(iterations):
    cfr_solver_1.evaluate_and_update_policy()
    cfr_solver_2.evaluate_and_update_policy()
    if i % print_freq == 0:
      conv_1 = exploitability.exploitability(game_1,
                                             cfr_solver_1.average_policy())
      conv_2 = exploitability.exploitability(game_2,
                                             cfr_solver_2.average_policy())

      print("Iteration {} exploitability of the  {} vs: {}".format(
          i, conv_1, conv_2))

  print("Final exploitability is {} vs {}".format(conv_1, conv_2))


game_1 = pyspiel.load_game("kuhn_poker",
                           {"players": 2})

universal_poker_kuhn_limit_2p = """\
GAMEDEF
limit
numPlayers = 2
numRounds = 1
blind = 1 1 
raiseSize = 1
firstPlayer = 1
maxRaises = 1
numSuits = 1
numRanks = 3
numHoleCards = 1
numBoardCards = 0
END GAMEDEF
"""
game_2 = pyspiel.load_game(
    "universal_poker",
    {"gamedef": universal_poker_kuhn_limit_2p})
compare_exploitability(game_1, game_2)

Iteration 0 exploitability of the  0.45833333333333326 vs: 0.45833333333333326
Iteration 1 exploitability of the  0.27083333333333337 vs: 0.27083333333333337
Iteration 2 exploitability of the  0.1944444444444444 vs: 0.1944444444444444
Iteration 3 exploitability of the  0.14062500000000003 vs: 0.14062500000000003
Iteration 4 exploitability of the  0.12138888888888891 vs: 0.12138888888888891
Iteration 5 exploitability of the  0.09672619047619041 vs: 0.09672619047619041
Iteration 6 exploitability of the  0.10796957671957674 vs: 0.10796957671957674
Iteration 7 exploitability of the  0.09045463963098324 vs: 0.09045463963098324
Iteration 8 exploitability of the  0.0775675533215757 vs: 0.0775675533215757
Iteration 9 exploitability of the  0.06869879381715754 vs: 0.06869879381715754
Final exploitability is 0.06869879381715754 vs 0.06869879381715754


In [6]:
game_1 = pyspiel.load_game("leduc_poker",
                           {"players": 2})
# Taken verbatim from the linked paper above: "In Leduc hold'em, the deck
# consists of two suits with three cards in each suit. There are two rounds.
# In the first round a single private card is dealt to each player. In the
# second round a single board card is revealed. There is a two-bet maximum,
# with raise amounts of 2 and 4 in the first and second round, respectively.
# Both players start the first round with 1 already in the pot.

universal_poker_leduc_limit_2p = """\
GAMEDEF
limit
numPlayers = 2
numRounds = 2
blind = 1 1
raiseSize = 1 1
firstPlayer = 1 1
maxRaises = 2 2
raiseSize = 2 4
numSuits = 2
numRanks = 3
numHoleCards = 1 0
numBoardCards = 0 1
END GAMEDEF
"""
game_2 = pyspiel.load_game(
    "universal_poker",
    {"gamedef": universal_poker_leduc_limit_2p})
compare_exploitability(game_1, game_2)

Iteration 0 exploitability of the  2.373611111111111 vs: 2.373611111111111
Iteration 1 exploitability of the  2.0613194444444445 vs: 2.0613194444444445
Iteration 2 exploitability of the  1.7988065869138403 vs: 1.7988065869138403
Iteration 3 exploitability of the  1.6590264691929169 vs: 1.6590264691929169
Iteration 4 exploitability of the  1.4713245596865683 vs: 1.4713245596865683
Iteration 5 exploitability of the  1.3046625366898728 vs: 1.3046625366898728
Iteration 6 exploitability of the  1.1565805178914506 vs: 1.1565805178914506
Iteration 7 exploitability of the  1.0447752045677787 vs: 1.0447752045677787
Iteration 8 exploitability of the  0.9617541410503099 vs: 0.9617541410503099
Iteration 9 exploitability of the  0.888578983168769 vs: 0.888578983168769
Final exploitability is 0.888578983168769 vs 0.888578983168769


In [7]:
game = "universal_poker"
out = "gametree.png"
prog = "dot"
group_infosets = False
group_terminal = False
verbose = False


def _zero_sum_node_decorator(state):
  """Custom node decorator that only shows the return of the first player."""
  attrs = treeviz.default_node_decorator(state)  # get default attributes
  if state.is_terminal():
    attrs["label"] = str(int(state.returns()[0]))
  return attrs


game = pyspiel.load_game(
    game, {"gamedef": universal_poker_kuhn_limit_2p})
game_type = game.get_type()

if game_type.dynamics != pyspiel.GameType.Dynamics.SEQUENTIAL:
  raise ValueError("Game must be sequential, not {}".format(game_type.dynamics))

if (game_type.utility == pyspiel.GameType.Utility.ZERO_SUM and
    game.num_players() == 2):
  gametree = treeviz.GameTree(
      game,
      node_decorator=_zero_sum_node_decorator,
      group_infosets=group_infosets,
      group_terminal=group_terminal)
else:
  gametree = treeviz.GameTree(game)  # use default decorators

if verbose:
  logging.info("Game tree:\n%s", gametree.to_string())

gametree.draw(out, prog=prog)

In [8]:
gametree.draw(out, prog=prog)